<a href="https://colab.research.google.com/github/subinmun1997/Kakao_chatbot/blob/master/Embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4MB 1.2MB/s 
     |████████████████████████████████| 460kB 42.9MB/s 
     |████████████████████████████████| 92kB 8.7MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [ ]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics={}
for word in nouns:
  if word not in dics.keys(): #이미 저장된 단어는 다시 사전에 저장하지 않기 위함.
    dics[word] = len(dics)
print(dics)

# 원-핫 인코딩
nb_classes = len(dics) # 원-핫 벡터 차원의 크기 결정
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets] # eye() 함수의 인자 크기대로 단위행렬을 반환하며, eye() 함수 뒤에 붙은 [targets]를 이용해 생성된 단위행렬의 순서를 단어 사전의 순서에 맞게 정렬해준다.
print(one_hot_targets)


['오늘', '날씨', '구름']
{'오늘': 0, '날씨': 1, '구름': 2}


In [10]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

def read_review_data(filename):
  with open(filename,'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]
    data = data[1:]
  return data

start = time.time()

print('1) 말뭉치 데이터 읽기 시작')
review_data = read_review_data('./ratings.txt')
print(len(review_data))
print('1) 말뭉치 데이터 읽기 완료 : ', time.time() - start)

print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료 : ', time.time() - start)

print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, size=200, window=4, hs=1, min_count=2, sg=1)
print('3) Word2Vec 모델 학습 완료 : ', time.time() - start)

print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료 : ', time.time() - start)

print('corpus_count : ', model.corpus_count)
print('corpus_total_words : ', model.corpus_total_words)


1) 말뭉치 데이터 읽기 시작
200000
1) 말뭉치 데이터 읽기 완료 :  0.668705940246582
2) 형태소에서 명사만 추출 시작
2) 형태소에서 명사만 추출 완료 :  132.4915473461151
3) Word2Vec 모델 학습 시작
3) Word2Vec 모델 학습 완료 :  182.4368908405304
4) 학습된 모델 저장 시작
4) 학습된 모델 저장 완료 :  184.13396549224854
corpus_count :  200000
corpus_total_words :  1076896


In [11]:
from gensim.models import Word2Vec

model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

print('사랑 : ', model.wv['사랑'])

print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))

print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))

corpus_total_words :  1076896
사랑 :  [-0.02827938  0.24900363 -0.17512138  0.6210899  -0.3314337  -0.15538982
  0.2909418   0.04805886  0.2435722  -0.3477459  -0.3149157   0.18925244
 -0.11251917  0.10376599  0.1921723  -0.10258646  0.04632262 -0.14551203
  0.23103102  0.04323523  0.0713856   0.1663442   0.1844679   0.23317616
 -0.25802755 -0.12826663  0.25148144 -0.13413396  0.24675131  0.03333859
  0.10242141 -0.07670885 -0.02714112 -0.15807842  0.38307896  0.26428843
  0.03128818  0.10059258  0.15300627 -0.4115303  -0.24056952  0.02982649
  0.16398722  0.1606651  -0.05351922  0.08212461  0.31000745 -0.12918471
  0.13638377 -0.01569323  0.18337773 -0.20521867 -0.06292526 -0.17809458
 -0.20780419  0.17584495 -0.28928357  0.06059166 -0.06441467  0.42964348
  0.14791939  0.05892763 -0.01454129  0.03029578  0.31331298 -0.09613602
  0.22997895 -0.09038579  0.0691096  -0.1895051   0.05484347 -0.00147987
 -0.31228215  0.120682   -0.11801404  0.10228568  0.18088444 -0.07271176
 -0.05614245  0